# 4 自动化程序优化

## 4.2 

In [1]:
from __future__ import annotations
import numpy as np
import tvm
from tvm import relax
from tvm.script import tir as T
from tvm.script import relax as R
from tvm import IRModule

In [2]:
import IPython


def code2html(code):
    """Helper function to use pygments to turn the code string into highlighted html."""
    import pygments
    from pygments.formatters import HtmlFormatter
    from pygments.lexers import Python3Lexer
    formatter = HtmlFormatter()
    html = pygments.highlight(code, Python3Lexer(), formatter)
    return "<style>%s</style>%s\n" % (formatter.get_style_defs(".highlight"), html)

## 4.3 review

In [3]:
# matmul
@tvm.script.ir_module
class MyModule():
    @T.prim_func
    def main(A: T.Buffer[(128, 128), 'float32'],
             B: T.Buffer[(128, 128), 'float32'],
             C: T.Buffer[(128, 128), 'float32']):
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i, j, k in T.grid(128, 128, 128):
            with T.block("C"):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                with T.init():
                    C[vi, vj] = T.float32(0)
                C[vi, vj] = C[vi, vj] + A[vi, vk] * B[vk, vj]
                

In [4]:
IPython.display.Code(MyModule.script(), language='python')

@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[(128, 128), "float32"], B: T.Buffer[(128, 128), "float32"], C: T.Buffer[(128, 128), "float32"]):
        # function attr dict
        T.func_attr({"tir.noalias": True, "global_symbol": "main"})
        # body
        # with T.block("root")
        for i, j, k in T.grid(128, 128, 128):
            with T.block("C"):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                T.reads(A[vi, vk], B[vk, vj])
                T.writes(C[vi, vj])
                with T.init():
                    C[vi, vj] = T.float32(0)
                C[vi, vj] = C[vi, vj] + A[vi, vk] * B[vk, vj]

In [5]:
dtype = 'float32'
a_np = np.random.rand(128, 128).astype(dtype)
b_np = np.random.rand(128, 128).astype(dtype)
c_mm = np.empty((128, 128), dtype=dtype)
np.matmul(a_np, b_np, out=c_mm)

array([[33.32014 , 33.514553, 32.598507, ..., 33.00354 , 31.367146,
        31.80453 ],
       [31.958197, 34.308468, 35.55669 , ..., 33.461643, 33.04884 ,
        33.053417],
       [32.362938, 36.79709 , 34.20112 , ..., 33.66086 , 35.539936,
        32.93146 ],
       ...,
       [32.826817, 30.674044, 32.781773, ..., 31.720518, 30.04795 ,
        31.067373],
       [31.326584, 31.578236, 31.222921, ..., 30.100452, 30.93048 ,
        26.668327],
       [30.931372, 32.27003 , 30.921936, ..., 30.705877, 29.459324,
        29.707888]], dtype=float32)

In [6]:
a_tvm = tvm.nd.array(a_np)
b_tvm = tvm.nd.array(b_np)
c_tvm = tvm.nd.empty((128, 128), dtype=dtype)
rt_lib = tvm.build(MyModule, target='llvm')
f_timer_before = rt_lib.time_evaluator('main', tvm.cpu(), number=1000)
print("Time cost of MyModule: %.3f ms" % (f_timer_before(a_tvm, b_tvm, c_tvm).mean * 1000))
np.testing.assert_allclose(c_tvm.numpy(), c_mm, rtol=1e-5)

Time cost of MyModule: 2.331 ms


In [7]:
def schedule_mm(sch: tvm.tir.Schedule, jfactor=4):
    block_C = sch.get_block("C", 'main')
    i, j, k = sch.get_loops(block_C)
    j0, j1 = sch.split(j, [None, jfactor])
    sch.reorder(j0, k, j1)

    sch.decompose_reduction(block_C, k)
    return sch

In [8]:
sch = tvm.tir.Schedule(MyModule)
sch = schedule_mm(sch, 4)
IPython.display.HTML(code2html(sch.mod.script()))

In [9]:
rt_lib_transformed = tvm.build(sch.mod, target='llvm')
f_timer_transformed = rt_lib_transformed.time_evaluator('main', tvm.cpu(), number=1000)
print("Time cost of MyModule=>schedule_mm: %.3f ms" % (f_timer_transformed(a_tvm, b_tvm, c_tvm).mean * 1000))
np.testing.assert_allclose(c_tvm.numpy(), c_mm, rtol=1e-5)

Time cost of MyModule=>schedule_mm: 1.680 ms


### 4.3.1 变换历史轨迹

历史轨迹与我们在 schedule_mm 中指定的变换一致。需要注意的一点是，历史轨迹加上原始程序一起，为我们提供了一种能够完全重新生成最终输出程序的方法。记住这一点，我们将在本章中使用历史轨迹作为检查变换的另一种方式。

In [10]:
sch.trace

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  l4, l5 = sch.split(loop=l2, factors=[None, 4], preserve_unit_iters=True)
  sch.reorder(l4, l3, l5)
  b6 = sch.decompose_reduction(block=b0, loop=l3)

## 4.4 随机调度变换（Stochastic Schedule Transformation）

In [11]:
# stochastic schedule transformation
def stochastic_schedule_mm(sch: tvm.tir.Schedule):
    block_C = sch.get_block('C', 'main')
    i, j, k = sch.get_loops(block_C)
    j_factors = sch.sample_perfect_tile(loop=j, n=2)
    j_0, j_1 = sch.split(j, j_factors)
    sch.reorder(j_0, k, j_1)
    sch.decompose_reduction(block_C, k)
    return sch

In [12]:
sch = tvm.tir.Schedule(MyModule)
sch = stochastic_schedule_mm(sch)

IPython.display.HTML(code2html(sch.mod.script()))

查看历史轨迹时，请密切注意 sample_perfect_tile 的 decision=[...] 部分。 它们对应于我们上次调用 stochastic_schedule_mm 时 sampling_perfect_tile 返回的值。

In [13]:
print(sch.trace)

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[32, 4])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)


In [14]:
rt_lib_stochastic = tvm.build(sch.mod, target='llvm')
f_timer_stochastic = rt_lib_stochastic.time_evaluator('main', tvm.cpu(), number=1000)
print("Time cost of MyModule=>stochastic_schedule_mm: %.3f ms" % (f_timer_stochastic(a_tvm, b_tvm, c_tvm).mean * 1000))
np.testing.assert_allclose(c_tvm.numpy(), c_mm, rtol=1e-5)



Time cost of MyModule=>stochastic_schedule_mm: 1.681 ms


### 4.4.1 深入研究随机变换

更深入地研究随机调度变换中发生的事情。我们可以发现它是原始确定性变换的简单泛化，包含两个附加元素：
1. 来自 sample_perfect_tile 的随机变量和我们在示例中未涵盖的其他采样操作。
2. 利用随机变量进行的后续变换操作。


In [15]:
sch = tvm.tir.Schedule(MyModule)
block_C = sch.get_block('C', 'main')
i, j, k = sch.get_loops(block_C)
j_factors = sch.sample_perfect_tile(j, 2)
type(j_factors[0]), j_factors[0]

(tvm.tir.expr.Var, v5)

j_factors 中的元素并不是实整数。相反，它们是指被采样的随机变量的符号变量

In [16]:
sch.trace

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[16, 8])

In [17]:
IPython.display.HTML(code2html(sch.mod.script()))

In [18]:
j_0, j_1 = sch.split(j, j_factors)
sch.reorder(j_0, k, j_1)
IPython.display.HTML(code2html(sch.mod.script()))

In [19]:
sch.trace

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[16, 8])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l6, l3, l7)

In [20]:
sch.decompose_reduction(block_C, k)

tir.BlockRV(0x5630959ba510)

In [21]:
IPython.display.HTML(code2html(sch.mod.script()))

## 4.5 随机变换搜索(基于模板的search)
stochastic_schedule_mm 创建了一个可能程序的搜索空间，具体取决于在每个采样步骤中做出的具体决定------> 希望能够指定一组**可能的程序**而不是一个程序

(stochastic_schedule_mm ----> space of possible programs)


In [22]:
def random_search(mod: tvm.IRModule, num_trials=5):
    best_sch = None
    best_result = None
    for i in range(num_trials):
        sch = tvm.tir.Schedule(mod)
        sch = stochastic_schedule_mm(sch)
        rt_lib = tvm.build(sch.mod, target='llvm')
        f_timer = rt_lib.time_evaluator('main', tvm.cpu(), number=1000)
        result = f_timer(a_tvm, b_tvm, c_tvm).mean
        
        print("=====Attempt %d, time-cost: %.3f ms====" % (i, result * 1000))
        print(sch.trace)

        if best_result is None or best_result > result:
            best_result = result
            best_sch = sch
        
    return best_sch

sch = random_search(MyModule, 5)


=====Attempt 0, time-cost: 1.838 ms====
# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[128, 1])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)
=====Attempt 1, time-cost: 1.584 ms====
# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[32, 4])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)
=====Attempt 2, time-cost: 1.620 ms====
# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(

In [23]:
sch.trace

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[32, 4])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)

**meta_schedule** 是支持搜索可能变换空间的命名空间。Meta-Schedule 在幕后做了很多额外的事情：

1. 跨越多个进程的并行基准测试。
2. 使用代价模型 (cost model) 来避免每次都进行基准测试。
3. 基于历史轨迹进行遗传搜索 (evolutionary search)，而不是每次都随机采样。

尽管有这些工具，但我们关键思想是保持不变的：1.使用随机变换来指定好程序的搜索空间，2.使用 ``tune_tir`` API 帮助在搜索空间内搜索并找到最优的调度变换。

In [24]:
from tvm import meta_schedule as ms
tuned_record = ms.tune_tir(MyModule, target='llvm -num-cores 1',
                        max_trials_global=64,
                        num_trials_per_iter=64,
                        space=ms.space_generator.ScheduleFn(stochastic_schedule_mm),
                        work_dir="./tune_tmp",
                        task_name='main')

2023-02-08 21:48:09 [INFO] [task_scheduler.cc:260] Task #0 has finished. Remaining task(s): 0


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,4194304,1,2.8253,1484.5611,1484.5611,5,Y


2023-02-08 21:48:09 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |    FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
------------------------------------------------------------------------------------------------------
  0 | main | 4194304 |      1 |         2.8253 |    1484.5611 |             1484.5611 |      5 |    Y 
------------------------------------------------------------------------------------------------------
Total trials: 5
Total latency (us): 1484.56



In [25]:
from tvm.meta_schedule.tir_integration import compile_tir
sch_tuned = compile_tir(tuned_record, MyModule, target="llvm")

In [26]:
IPython.display.HTML(code2html(sch_tuned.mod.script()))

In [27]:
sch_tuned.trace

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  l1, l2, l3 = sch.get_loops(block=b0)
  v4, v5 = sch.sample_perfect_tile(loop=l2, n=2, max_innermost_factor=16, decision=[8, 16])
  l6, l7 = sch.split(loop=l2, factors=[v4, v5], preserve_unit_iters=True)
  sch.reorder(l6, l3, l7)
  b8 = sch.decompose_reduction(block=b0, loop=l3)
  sch.enter_postproc()

In [28]:
lib = tvm.build(sch_tuned.mod, target='llvm')
f_timer_after = lib.time_evaluator('main', tvm.cpu(), number=1000)
print("Time cost of MyModule after tuning: %.3f ms" % (f_timer_after(a_tvm, b_tvm, c_tvm).mean * 1000))

Time cost of MyModule after tuning: 1.209 ms


### 4.5.1 利用默认自动调度(auto-schedule)

上一节中，我们展示了如何使用我们精心设计的随机变换来优化 IRModule 的计算(基于手动设计模板)。Meta-Schedule 带有内置通用随机变换集合，能够适用于广泛的 TensorIR 计算。这种方法也称为自动调度 (auto-scheduling)，因为搜索空间是由系统生成的

In [29]:
tuned_record_ansor = ms.tune_tir(MyModule, target='llvm --num-cores=1',
                                 max_trials_global=64,
                                 num_trials_per_iter=64,
                                 work_dir="./tune_tmp",
                                 task_name='main')

2023-02-08 21:49:07 [INFO] [task_scheduler.cc:260] Task #0 has finished. Remaining task(s): 0


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,4194304,1,240.4021,17.4470,17.4470,64,Y


2023-02-08 21:49:07 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |    FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
------------------------------------------------------------------------------------------------------
  0 | main | 4194304 |      1 |       240.4021 |      17.4470 |               17.4470 |     64 |    Y 
------------------------------------------------------------------------------------------------------
Total trials: 64
Total latency (us): 17.447



In [31]:
sch_tuned_ansor = compile_tir(tuned_record_ansor, MyModule, target='llvm')
lib_ansor = tvm.build(sch_tuned_ansor.mod, target='llvm')

f_timer_ansor = lib_ansor.time_evaluator('main', tvm.cpu(), number=1000)
print("Time cost of MyModule after ansor tuning: %.3f ms" % (f_timer_ansor(a_tvm, b_tvm, c_tvm).mean * 1000))


Time cost of MyModule after ansor tuning: 0.033 ms


In [34]:
IPython.display.HTML(code2html(sch_tuned_ansor.mod.script()))

结果比我们的原始代码快得多。我们可以查看历史轨迹和最终代码。就本章而言，你不需要了解所有变换。在高层次的理解中，历史轨迹包含：

1. 更多级的循环转换
2. 中间计算的矢量化
3. 并行化和循环展开

In [36]:
np.testing.assert_allclose(c_tvm.numpy(), c_mm, rtol=1e-5)
sch_tuned_ansor.trace

# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="C", func_name="main")
  b1 = sch.get_block(name="root", func_name="main")
  sch.annotate(block_or_loop=b0, ann_key="meta_schedule.tiling_structure", ann_val="SSRSRS")
  l2, l3, l4 = sch.get_loops(block=b0)
  v5, v6, v7, v8 = sch.sample_perfect_tile(loop=l2, n=4, max_innermost_factor=64, decision=[2, 2, 16, 2])
  l9, l10, l11, l12 = sch.split(loop=l2, factors=[v5, v6, v7, v8], preserve_unit_iters=True)
  v13, v14, v15, v16 = sch.sample_perfect_tile(loop=l3, n=4, max_innermost_factor=64, decision=[2, 2, 8, 4])
  l17, l18, l19, l20 = sch.split(loop=l3, factors=[v13, v14, v15, v16], preserve_unit_iters=True)
  v21, v22 = sch.sample_perfect_tile(loop=l4, n=2, max_innermost_factor=64, decision=[64, 2])
  l23, l24 = sch.split(loop=l4, factors=[v21, v22], preserve_unit_iters=True)
  sch.reorder(l9, l17, l10, l18, l23, l11, l19, l24, l12, l20)
  b25 = sch.cache_write(block=b0, write_buffer_index=0, stor

### 4.5.2 章节回顾
随机调度允许我们表示“可能的变换是什么”。

Meta-Schedule 的 tune_tir API 帮助我们在搜索空间内找到一个好的解决方案。

Meta-Schedule 带有一组默认的内置随机变换，涵盖了广泛的搜索空间。